<a href="https://colab.research.google.com/github/SAAD-Nahri/Examen_Master_MAAIS_SAAD_NAHRI/blob/main/LSTM_CodeTrading_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
import yfinance as yf

# Download Boeing's stock data
#sp500_data = yf.download('BA', start='2010-01-01', end='2024-01-01')

sp500_data = yf.download('^GSPC', start='2010-01-01', end='2024-01-01')


In [3]:

log_returns = np.log(sp500_data['Close'] / sp500_data['Close'].shift(1)).dropna()
from scipy.signal import savgol_filter
denoised_log_returns = savgol_filter(log_returns, window_length=11, polyorder=2)
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(denoised_log_returns).reshape(-1,1))
print(closedf.shape)

(3521, 1)


In [33]:
training_size=int(len(closedf)*0.70)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)\
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

train_data:  (2464, 1)
test_data:  (1057, 1)
X_train:  (2448, 15)
y_train:  (2448,)
X_test:  (1041, 15)
y_test (1041,)


In [91]:
import numpy as np

class LinearRegression:
    def __init__(self, lambda_reg=1e-4, nbPointsToIgnore=100):
        self.lambda_reg = lambda_reg
        self.nbPointsToIgnore = nbPointsToIgnore
        self.Wout = None  # To store the trained output weight matrix

    def train(self, u, y):
        u = u.T
        S = np.vstack([np.ones((1, u.shape[1])), u[:,:]])
        D = y[:]
        I = np.eye(S.shape[0])
        self.Wout = D @ S.T @ np.linalg.pinv(S @ S.T + self.lambda_reg * I)

    def predict(self, u_test):
        u_test = u_test.T
        if self.Wout is None:
            raise ValueError("Model has not been trained yet.")
        S_test = np.vstack([np.ones((1, u_test.shape[1])), u_test])
        print(S_test.shape)
        y_pred = self.Wout @ S_test
        return y_pred



In [92]:
model = LinearRegression()

# Train the model
model.train(X_train, y_train)
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)


train_predict = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1,1))
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

(16, 2448)
(16, 1041)
Train data RMSE:  0.001815731031690747
Train data MSE:  3.296879179444744e-06
Test data MAE:  0.0013020796497533842
-------------------------------------------------------------------------------------
Test data RMSE:  0.0027851475360665774
Test data MSE:  7.757046797657728e-06
Test data MAE:  0.0018323634516613392


In [93]:
X_train1 =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test1 = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train1.shape)
print("X_test: ", X_test1.shape)

X_train:  (2448, 15, 1)
X_test:  (1041, 15, 1)


In [96]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.fit(X_train1,y_train,validation_data=(X_test1,y_test),epochs=20,batch_size=5,verbose=1)

Epoch 1/20
490/490 [==============================] - 20s 25ms/step - loss: 0.0091 - val_loss: 0.0081
Epoch 2/20
490/490 [==============================] - 11s 22ms/step - loss: 0.0043 - val_loss: 0.0083
Epoch 3/20
490/490 [==============================] - 10s 20ms/step - loss: 0.0043 - val_loss: 0.0084
Epoch 4/20
490/490 [==============================] - 13s 26ms/step - loss: 0.0042 - val_loss: 0.0076
Epoch 5/20
490/490 [==============================] - 10s 20ms/step - loss: 0.0034 - val_loss: 0.0043
Epoch 6/20
490/490 [==============================] - 10s 21ms/step - loss: 0.0017 - val_loss: 0.0025
Epoch 7/20
490/490 [==============================] - 10s 21ms/step - loss: 0.0013 - val_loss: 0.0024
Epoch 8/20
490/490 [==============================] - 11s 22ms/step - loss: 0.0012 - val_loss: 0.0022
Epoch 9/20
490/490 [==============================] - 10s 21ms/step - loss: 0.0012 - val_loss: 0.0022
Epoch 10/20
490/490 [==============================] - 10s 20ms/step - loss: 0.001

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

In [ ]:

import numpy as np



class RRBFN:
    def __init__(self, input_size, reservoir_size, output_size, alpha, beta, lambda_reg=1e-8):
        self.Nu = input_size
        self.Nx = reservoir_size
        self.Ny = output_size
        self.alpha = alpha
        self.beta = beta
        self.lambda_reg = lambda_reg
        self.Win = np.random.uniform(-1, 1, (self.Nx, self.Nu))
        self.W = np.random.uniform(-1, 1, (self.Nx, self.Nx))
        self.Wout = None
        self.X_final None


    def train(self, U, Y, ignore_initial=100):
        U = U.T
        nbDataPoints = U.shape[1]
        X = np.zeros((self.Nx, nbDataPoints))
        Z = np.zeros((1+self.Nu+self.Nx, nbDataPoints))
        for i in range(1, nbDataPoints):
            for j in range(1, self.Nx):
                squared_norm_u = np.sum(((self.Win[j,:]).T - U[:, i]) ** 2)
                squared_norm_x = np.sum(((self.W[j,:]).T - X[:, i-1]) ** 2)
                X[j, i] = np.exp(-self.alpha * squared_norm_u - self.beta * squared_norm_x)
            Z[:,i] = np.hstack([[1,], U[:,i], X[:,i]])
        # Save the final state of X to be used in prediction
        self.X_final = X[:, -1]
        S = Z[:,:]
        D = Y[:]
        self.Wout = D @ S.T @ np.linalg.inv(S @ S.T + self.lambda_reg * np.eye(S.shape[0]))

    def predict(self, U):
        U = U.T
        nbDataPoints = U.shape[1]
        X = np.zeros((self.Nx, nbDataPoints))
        Z = np.zeros((1 + self.Nu + self.Nx, nbDataPoints))
        # Start with the last state of X from training
        X[:, 0] = self.X_final
        for i in range(1, nbDataPoints):
            for j in range(1,self.Nx):
                squared_norm_u = np.sum(((self.Win[j,:]).T - U[:, i]) ** 2)
                squared_norm_x = np.sum(((self.W[j,:]).T - X[:, i-1]) ** 2)
                X[j, i] = np.exp(-self.alpha * squared_norm_u - self.beta * squared_norm_x)
            Z[:,i] = np.hstack([1, U[:,i], X[:,i]])
        return self.Wout @ Z


In [ ]:
# Initialize model parameters
input_size = 15  # Since we're using log returns directly
reservoir_size = 100  # Example size, should be tuned
output_size = 1  # Predicting one step ahead
alpha = 0.2  # Example, should be tuned
beta = 0.8  # Example, should be tuned
model = RRBFN(input_size, reservoir_size, output_size, alpha, beta)

# Train the model
model.train(X_train, y_train)
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1,1))
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

In [84]:
import numpy as np

class RBFN:
    def __init__(self, input_size, num_neurons, eta=0.01):
        self.input_size = input_size  # Number of input features
        self.num_neurons = num_neurons  # Number of neurons in the hidden layer
        self.eta = eta  # Learning rate
        self.centers = np.random.uniform(low=0, high=1, size=(num_neurons, input_size))  # Centers of the RBFs
        self.betas = np.random.uniform(low=0.1, high=0.3, size=num_neurons)  # Betas (spread) of the RBFs
        self.weights = np.random.uniform(low=-0.1, high=0.1, size=num_neurons)  # Output weights

    def _rbf(self, input_vector, center, beta):
        return np.exp(-np.linalg.norm(input_vector - center)**2 / (2 * beta**2))

    def train(self, U, Y, max_iterations=500, tolerance=0.009):
        U = U.T
        nbDataPoints = U.shape[1]
        error = 1e5
        error1 = 1e5
        k = 1

        while error > tolerance and k < max_iterations:
            for j in range(nbDataPoints):
                phi = np.array([self._rbf(U[:, j], self.centers[i], self.betas[i]) for i in range(self.num_neurons)])
                y_pred = np.dot(self.weights, phi)
                e = y_pred - Y[j]

                for i in range(self.num_neurons):
                    # Update rules as per the algorithm
                    self.weights[i] -= self.eta * e * phi[i]
                    self.centers[i] -= self.eta * e * self.weights[i] * phi[i] * (U[:, j] - self.centers[i]) / self.betas[i]**2
                    self.betas[i] -= self.eta * e * self.weights[i] * phi[i] * np.linalg.norm(U[:, j] - self.centers[i]) / self.betas[i]**3

            # Recalculate the total error for convergence checking
            y_hats = np.array([np.dot(self.weights, [self._rbf(U[:, j], self.centers[i], self.betas[i]) for i in range(self.num_neurons)]) for j in range(nbDataPoints)])
            error = np.sqrt(np.mean((Y - y_hats)**2))
            error1 = np.sqrt(np.mean((scaler.inverse_transform(Y.reshape(-1,1)) - scaler.inverse_transform(y_hats.reshape(-1,1)))**2))
            k += 1
            print(error,';;;;;;;;;;;;;;;;;;;;;;;1')
            print(error1,';;;;;;;;;;;;;;;;;;;;;;;1')
            print(k,';;;;;;;;;;;;;;;;;;;;;;;1')

    def predict(self, U):
        U = U.T
        """ Predict the output for given input using trained RBFN """
        predictions = np.array([np.dot(self.weights, [self._rbf(U[:, j], self.centers[i], self.betas[i]) for i in range(self.num_neurons)]) for j in range(U.shape[1])])
        return predictions




In [85]:

model = RBFN(input_size=15, num_neurons=100)


In [ ]:
model.train(X_train, y_train)

In [46]:

train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1,1))
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  0.004676374435994357
Train data MSE:  2.186847786562154e-05
Test data MAE:  0.0034813971043036484
-------------------------------------------------------------------------------------
Test data RMSE:  0.0069405409532716355
Test data MSE:  4.817110872404075e-05
Test data MAE:  0.0049191348585272895


In [90]:

def naive_forecast(X_train):
    return X_train[:, -1]

train_predict=naive_forecast(X_train)
test_predict=naive_forecast(X_test)

train_predict = scaler.inverse_transform(train_predict.reshape(-1,1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1,1))
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))


Train data RMSE:  0.0023214041632212065
Train data MSE:  5.3889172890207495e-06
Test data MAE:  0.00165981207101986
-------------------------------------------------------------------------------------
Test data RMSE:  0.0033253828362837827
Test data MSE:  1.1058171007850775e-05
Test data MAE:  0.0022334910445425665
